In [ ]:
#load images
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, optimizers, callbacks
import matplotlib.pyplot as plt 
from PIL import Image
import os
%matplotlib inline

### Load images

In [ ]:
batch_size = 16
train_data_dir = "raw_data/training-data-exp/"
#test_data_dir = "raw_data/test-data-binary/"

train_ds = image_dataset_from_directory(
  train_data_dir,
  labels = "inferred",
  label_mode = "binary",
  #class_names = ['clean','dust'],  
  seed=123,
  image_size=(225, 225),
  batch_size=batch_size,
  validation_split=0.2,
  subset='both'
) 

train_data = train_ds[0]
val_data = train_ds[1]

In [ ]:
class_names = val_data.class_names
class_names

### Binary model: Clean vs Snow

In [ ]:
# We'll use a model with the same convolutional layers, but we'll add Augmentation layers before that

model_clean_snow = Sequential()

model_clean_snow.add(layers.Rescaling(1./255, input_shape = (225, 225, 3)))

# Data Augmentation Layers

model_clean_snow.add(layers.RandomFlip("horizontal"))
model_clean_snow.add(layers.RandomZoom(0.1))
model_clean_snow.add(layers.RandomTranslation(0.2, 0.2))
model_clean_snow.add(layers.RandomRotation(0.1))


# Convolutional Layers

model_clean_snow.add(layers.Conv2D(filters = 32, kernel_size = (3,3), activation="relu", padding = "same"))
model_clean_snow.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )


model_clean_snow.add(layers.Conv2D(filters = 32, kernel_size = (3,3), input_shape = (225, 225, 3), activation="relu", padding = "same"))
model_clean_snow.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )


model_clean_snow.add(layers.Conv2D(filters = 64, kernel_size = (3,3), input_shape = (225, 225, 3), activation="relu", padding = "same"))
model_clean_snow.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )

model_clean_snow.add(layers.Conv2D(filters = 128, kernel_size = (3,3), input_shape = (225, 225, 3), activation="relu", padding = "same"))
model_clean_snow.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )

model_clean_snow.add(layers.Flatten())

model_clean_snow.add(layers.Dense(64, activation="relu"))

model_clean_snow.add(layers.Dropout(0.5))

model_clean_snow.add(layers.Dense(1, activation="sigmoid"))

In [ ]:
adam = optimizers.Adam(learning_rate = 0.001)

model_clean_snow.compile(loss= 'binary_crossentropy', #'categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [ ]:
MODEL = "model_clean_damage"

modelCheckpoint = callbacks.ModelCheckpoint("{}.h5".format(MODEL), monitor="val_loss", verbose=0, save_best_only=True)

LRreducer = callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience=3, verbose=1, min_lr=0)

EarlyStopper = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

In [ ]:
%%time

history_clean_snow = model_clean_snow.fit(
        train_data,
        epochs=30,
        validation_data=val_data,
        callbacks = [modelCheckpoint, LRreducer, EarlyStopper]
        )

### Pred tests

In [ ]:
def predictImage_binary(url, model):

  # Takes an imafe and a model

  img = url
  img = img_to_array(img)
  img = img.reshape((-1, 225, 225, 3))
  res = model.predict(img)
  print(f"Probabilities: ")
  print(f"{res[0]}")  
  res = model.predict(img)[0][0]
  if(res < 0.5):
    pred_class = "clean"
    prob = 1-res
  if(res >= 0.5):
    pred_class = "damage"
    prob = res

  print("Class : ", pred_class)
  print("probability = ",prob)

In [ ]:
def predictImage_multiclass(url, model):

  # Takes an image and a model

  img = url
  img = img_to_array(img)
  img = img.reshape((-1, 225, 225, 3))
  res = model.predict(img)
  print(f"Probabilities: ")
  names_of_classes = class_names
  print(f"{names_of_classes}")
  print(f"{res[0]}")
  print(f"Result: {names_of_classes[find_index_of_max_element(res[0].tolist())]}")  
  return res 

In [ ]:
img_clean_1 = load_img(f"raw_data/training-data/clean/Cleaan (4).jpeg", target_size=(225, 225))
predictImage_binary(img_clean_1,model_clean_snow)
plt.imshow(img_clean_1);

In [ ]:
img_clean_2 = load_img(f"raw_data/training-data/clean/Cleaan (15).jpeg", target_size=(225, 225))
predictImage_binary(img_clean_2,model_clean_snow)
plt.imshow(img_clean_2);

In [ ]:
img_clean_2 = load_img(f"raw_data/training-data/solar/Solar (15).jpeg", target_size=(225, 225))
predictImage_binary(img_clean_2,model_clean_snow)
plt.imshow(img_clean_2);

In [ ]:
img_clean_2 = load_img(f"raw_data/training-data/solar/Solar (21).jpeg", target_size=(225, 225))
predictImage_binary(img_clean_2,model_clean_snow)
plt.imshow(img_clean_2);

In [ ]:
img_clean_2 = load_img(f"raw_data/training-data/solar/Solar (28).jpeg", target_size=(225, 225))
predictImage_binary(img_clean_2,model_clean_snow)
plt.imshow(img_clean_2);

## Precision, Recall

In [ ]:
import numpy as np
from tensorflow.math import confusion_matrix
from sklearn.metrics import classification_report


predictions = np.array([])
probabilities = np.array([])
labels =  np.array([])

for x, y in val_data:
    
  proba = model_clean_snow.predict(x)
  pred = np.where(proba < 0.5, 0, 1).reshape(len(y))
    
  predictions = np.concatenate([predictions, pred])

  label =  np.squeeze(y) #y.reshape(len(pred))
  proba_damage = np.squeeze(1- proba)  
  prob = proba.reshape(len(y))

  probabilities = np.concatenate([probabilities, proba_damage])  

  labels = np.concatenate([labels, label])  #np.argmax(y.numpy()

report = classification_report(labels, predictions, target_names=class_names)

print(report)

In [ ]:
confusion_matrix(labels=labels, predictions=predictions).numpy()

In [ ]:
from sklearn.metrics import precision_recall_curve
import pandas as pd

In [ ]:


precision, recall, threshold = precision_recall_curve(labels, probabilities)
scores = pd.DataFrame({'threshold':threshold,
                       'precision': precision[:-1],
                       'recall':recall[:-1]}) # Store in a dataframe
scores


In [ ]:
plt.plot(scores['recall'],scores['precision'])
plt.ylabel('precision')
plt.xlabel('recall')

In [ ]:
new_threshold = scores[scores['precision'] >= 0.4].threshold.min()
new_threshold

In [ ]:
def custom_predict(X, custom_threshold, model):
    probability = model.predict(X) # Get likelihood of each sample being classified as 0 or 1
    proba_damage = probability
    #print(modified_array)
    #more_5y_probs = probability[:, 1] # Only keep expensive likelihoods (1) 
    return (proba_damage > custom_threshold).astype(int) # Boolean outcome converted to 0 or 1

pred_thres = np.array([])
for x, y in val_data:    
    updated_preds = custom_predict(X=x, custom_threshold=new_threshold, model=model_clean_snow) # Update prediction
    updated_preds = np.squeeze(updated_preds)
    pred_thres = np.concatenate([pred_thres, updated_preds]) 

## Threshold adjustment

In [ ]:
confusion_matrix(labels=labels, predictions=pred_thres).numpy()

In [ ]:

report = classification_report(labels, pred_thres, target_names=class_names)

print(report)

In [ ]:
#for label, pred in zip(labels, pred_thres):
#    print(f'label: {label}, pred: {pred}')

### Combining models

In [ ]:
#Code for combining models

def combine_models(model1,model2,image)
    prediction = model1.predict(image)

    if prediction!='dirt':
        prediction = model2(image)
    return prediction


    

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
model1 = load_model('model_filename.h5')

# Now you can use the loaded model for predictions or further training
